# APLICACIONES EN CIENCIAS DE COMPUTACION

## Laboratorio 4:  Búsqueda Local (Hill climbing, Simulated annealing y Beam search) 
Indicaciones previas:
- Las respuestas deben tener un buen fundamento teórico, se realizarán descuentos en el puntaje a respuestas que no contesten a lo solicitado
- Cualquier indicio de plagio resultará en la anulación de la prueba.

La tarea de este laboratorio consiste en comparar métodos de busqueda local para la resolución de la N-Reinas.<br>Al final de este notebook se encuentran las preguntas que serán evaluadas en este laboratorio. 

**Usted deberá completar el código en las secciones indicadas con "MODIFICAR" y "COMPLETAR"**

Ejemplo de representacion de estado (tablero nqueens):
<img src="state_nqueens.png">


In [16]:
import numpy as np
from collections import defaultdict
from collections import Counter
from random import randrange
import random
from time import time
from random import shuffle, random, sample, randint, choice, uniform
from copy import deepcopy
import math
from math import exp
import sys

### Clase <b>SearchProblem</b>

Esta es una clase abstracta para definir problemas de busqueda. Se debe hacer subclases que implementen los metodos de las acciones, resultados, test de objetivo y el costo de camino. Entonces se puede instanciar las subclases y resolverlos con varias funciones de busqueda.

In [17]:
# ESTA CELDA NO NECESITA SER MODIFICADA  
class SearchProblem:
    def __init__(self, initial=None):
        """Initialize a search problem with a initial state"""
        pass

    def initial(self):
        """Return default initial state of the search problem"""
        pass

    def value(self, state):
        """Return the value of the state. This is the objective function to be optimized"""
        pass

    def neighborhood(self, state):
        """Return the neighboring states of the given state"""
        pass

    def random_neighbor(self, state):
        """Return a random neighbor of the neighborhood of the state (used in simulated annealing)"""
        return choice(self.neighborhood(state))



### Clase <b>NQueensSearch</b>

La clase NQueenSearch implementa concretamente el problema del  tablero de las NQueens. Esta se representa mediante una tupla, en la cual se indica la posición de cada reina Q. Además, incluye el metodo value() para conocer la cantidad de pares de reinas atacadas mutuamente.

In [18]:
# ESTA CELDA NO NECESITA SER MODIFICADA
class NQueensSearch(SearchProblem):
    '''
    State: (QueenCoords)
    '''
    def __init__(self, filename,N=8):
        self.N = N
        self.file = filename

    def initial(self):
        """ Lee el archivo para retornar una tupla con las posiciones de cada Reinas del tablero """
        fd=open(self.file,"r+")
        puzzle=eval(fd.readline())
        board=[]
        for i in puzzle:
            board.append(i)
        return tuple(board)
    
    def value(self, state):
        """ Retorna número de pares de Queens que se atacan mutuamente. Se recorre State: (QueenCoords) 
            para agregar el ataque de cada reina, tanto en sus diagonales como en su misma columna. Luego, 
            se recorre cada collection creado para incrementar el número de pares de reinas atacadas (clashes)"""
        columnQ, diag1Q, diag2Q = [Counter() for i in range(3)]
        #En este caso state = (6,0,6,5,4,6,5,0) 
        for row, col in enumerate(state):
            columnQ[col] += 1       
            diag1Q[row - col] += 1  
            diag2Q[row + col] += 1
        clashes = 0
        for cnt in [columnQ, diag1Q, diag2Q]:
            for key in cnt:
                clashes += cnt[key] * (cnt[key] - 1) // 2
        return clashes

    def neighborhood(self, state):
        """ Crea nuevos tableros vecinos, diferentes al original """
        neighborhood = []
        for row in range(self.N):  # por cada fila
            for col in range(self.N):  # por cada columna
                # genera tablero vecino moviendo reyna de fila row a la columna col (siempre que no exita reyna en (row,col))    
                if col != state[row]: 
                    neighbor = list(state)
                    neighbor[row] = col
                    neighborhood.append(tuple(neighbor))
        return neighborhood

    def random_neighbor(self, state):
        """ Genera un tablero vecino de manera aleatoria, a partir del tablero original pasado (state)"""
        row = randrange(self.N)  # escoge una fila aleatoriamente
        col = choice( [i for i in range(self.N) if i!=state[row]] ) # escoge una columna aleatoria diferente de donde esta la reyna en esa fila
        neighbor = list(state)
        neighbor[row] = col
        return tuple(neighbor)

### Funciones utilitarias para manejar el tablero NQueens</b>
Estas son funciones utilitarias para mostrar el tablero 

In [19]:
# ESTA CELDA NO NECESITA SER MODIFICADA
n=8
def make_board(result):
    board = []
    espacio =['_']*(n+1)
    espacio[0]=' '
    board.append(str().join(espacio))
    for col in result:
        line = ['*'] * (n+2)
        line[0]='|'
        line[col+1] = 'Q'
        line[n+1]='|'
        board.append(str().join(line))
    board.append(str().join(espacio))
    return board

def printBoard(board):
    charlist = list(map(list, board))
    for line in charlist:
        print(" ".join(line))
def ShowBoard(current):
    """ Muestra la distribución de Queens(Q) en el tablero.   """
    board = make_board(current)
    printBoard(board)

## <b>Algoritmos de Búsqueda Local</b> 

### <b>Hill-climbing </b> 

Implementación del algoritmo para la resolución de NQueenSearchs

In [20]:
################################## MODIFICAR ###################################
# HACER LAS MODIFICACIONES NECESARIAS PARA QUE ENCUENTRE TABLERO DE MINIMO SCORE

def hill_climbing(problem, maxIter=1000):
    count=0  # contador de iteraciones desde que se encuentra el 1er tablero solucion 
    current = problem.initial()  # lee el archivo del tablero inicial
    current_score = problem.value(current) # evalua tablero inicial
    
    # muestra tablero inicial
    print('Hill Climbing intentará resolver el siguiente tablero NQueens:')
    ShowBoard(current)  
    print()
        
    start_time = time()  # inicia el contador de tiempo
    t = 0
    while (t < maxIter):
        if (t % 100 == 0): 
            print('Iteration {},\tCurrent score  = {}'.format(t, problem.value(current)))
            
        neighborhood = problem.neighborhood(current)
        if not neighborhood:
            break
            
        neighborhood_scores = []
        for i in range(len(neighborhood)): # evalua cada tablero vecino
            neighborhood_scores.append(problem.value(neighborhood[i]) )
        index_best_neighbor = np.argmin(neighborhood_scores)
        #ORIGINAL index_best_neighbor = np.argmax(neighborhood_scores)  obtiene el indice del mejor tablero
       
        if neighborhood_scores[index_best_neighbor] <= current_score:  # si el mejor vecino es mejor que el tablero current
            #ORIGINAL ------>  if neighborhood_scores[index_best_neighbor] >= current_score: 
            current_score = neighborhood_scores[index_best_neighbor]
            current = deepcopy(neighborhood[index_best_neighbor])
        
        if problem.value(current) == 0:  # si es tablero solucion
            count += 1  # aumenta contador de tableros solucion encontrados
        
        t += 1
    end_time = time()  # stop el contador de tiempo
    print('\nN° de tableros solución: %2d en %d iteraciones \nRunning time: %f'%(count,maxIter , end_time-start_time))
    print('Mejor tablero solución hallado con score {}'.format(problem.value(current)))
    ShowBoard(current)  # muestra el tablero final


### <b>Simulating Annealing</b> 
Implementación del algoritmo para la resolución de NQueenSearchs

In [21]:
################################## MODIFICAR ###################################
# HACER LAS MODIFICACIONES NECESARIAS PARA QUE ENCUENTRE TABLERO DE MINIMO SCORE
def simulated_annealing(problem, T0, DR, maxIter):
    """ Simulating annealing solver. 
           T0: Initial temperature
           DR: The decay rate of the schedule function: Ti = T0*(DR)^i (Ti is the temperature at iteration i). 
               For efficiecy the schedule function is implemented as: Ti = T(i-1)*DR
      maxIter: The maximum number of iterations
    """
    count=0   # contador de iteraciones desde que se encuentra el 1er tablero solucion 
    current = problem.initial()  # lee el archivo del tablero inicial
    current_score = problem.value(current)*-1    # evalua tablero inicial
    # ORIGINAL ----> current_score = problem.value(current) 

    # muestra tablero inicial    
    print('Simulated Annealing intentará resolver el siguiente tablero NQueens: ')
    ShowBoard(current)  # muestra tablero inicial
    print()    
    
    start_time = time()  # inicia el contador de tiempo
    best_score = current_score
    T=T0  # inicia temperatura en temperatura inicial
    t = 0
    while (t < maxIter):
        if (t % 10000 == 0): 
            print('Iteration {},\tTemperaure = {},\tBest score = {},\tCurrent score = {}'.format(t, T,best_score, current_score))

        neighbor      = problem.random_neighbor(current)
        neighborScore = problem.value(neighbor)*-1  # evalua tablero vecino
        # ORIGINAL ----> neighborScore = problem.value(neighbor)
        delta = float(neighborScore - current_score)  # diferencia entre el score del tablero vecino con respecto al actual
            
        if (delta > 0):   # si el tablero vecino generado es mejor que el actual, se acepta
            
            current = neighbor
            current_score = neighborScore 
        elif ( random() < exp((delta/T)) ):  # si el tablero generado es peor, se acepta con probability  exp((delta/T))
            current = neighbor
            current_score = neighborScore 
            
        if (current_score > best_score):  # si el tablero actual es mejor que el mejor tablero encontado hasta ahora
          
            best_board = deepcopy(current)
            best_score = current_score
                
        if current_score == 0:  # si es tablero solucion   
            best_board = current
            best_score = current_score
            if count==0: 
                best_iteracion = t   # iteracion donde se encontro el 1er tablero solucion
            count += 1  # aumenta contador de tableros solucion encontrados
            
        T = T*DR   # aplica decaimiento de temperatura
        t += 1
    end_time = time()  # stop del contador de tiempo
    
    if best_score == 0:
        print ("\nSA encontro tablero solucion en iteracion = {} de {} iteraciones".format(best_iteracion, t))
        
    else:
        print("\nSA no encontró tablero solucion!. Este es el mejor tablero encontrado con score={}:".format(best_score))
    
    print("N° de tableros solución: %2d en %d iteraciones \nRunning time: %f"%(count, t , end_time-start_time))
    ShowBoard(best_board)  # muestra el mejor tablero

## <b> Experimentación con los algoritmos de Búsqueda</b> 

In [22]:
# ESTA CELDA NO NECESITA SER MODIFICADA
""" Carga un tablero de archivo en disco e instancia el problema de busqueda  """
problem = NQueensSearch("queens.txt")
print("8-Queens Original")
ShowBoard(problem.initial())

8-Queens Original
  _ _ _ _ _ _ _ _
| * * * * * * Q * |
| Q * * * * * * * |
| * * * * * * Q * |
| * * * * * Q * * |
| * * * * Q * * * |
| * * * * * * Q * |
| * * * * * Q * * |
| Q * * * * * * * |
  _ _ _ _ _ _ _ _


### Hill Climbing

Llama a Hill Climbing para resolver el tablero 'queens.txt' con numero de iteraciones maxIter=1000

In [23]:
###### COMPLETAR PRUEBA #########
problem = NQueensSearch("queens.txt")
print("8-Queens Hill Climbing")
hill_climbing(problem,1000)

8-Queens Hill Climbing
Hill Climbing intentará resolver el siguiente tablero NQueens:
  _ _ _ _ _ _ _ _
| * * * * * * Q * |
| Q * * * * * * * |
| * * * * * * Q * |
| * * * * * Q * * |
| * * * * Q * * * |
| * * * * * * Q * |
| * * * * * Q * * |
| Q * * * * * * * |
  _ _ _ _ _ _ _ _

Iteration 0,	Current score  = 10
Iteration 100,	Current score  = 1
Iteration 200,	Current score  = 1
Iteration 300,	Current score  = 1
Iteration 400,	Current score  = 1
Iteration 500,	Current score  = 1
Iteration 600,	Current score  = 1
Iteration 700,	Current score  = 1
Iteration 800,	Current score  = 1
Iteration 900,	Current score  = 1

N° de tableros solución:  0 en 1000 iteraciones 
Running time: 2.029613
Mejor tablero solución hallado con score 1
  _ _ _ _ _ _ _ _
| * * Q * * * * * |
| * * * * * Q * * |
| * * * * * * * Q |
| * Q * * * * * * |
| * * * * Q * * * |
| * * * * * * Q * |
| * * * Q * * * * |
| Q * * * * * * * |
  _ _ _ _ _ _ _ _


### Simulated Annealing

In [24]:
###### COMPLETAR PRUEBA #########
problem = NQueensSearch("queens.txt")
print("8-Queens Simulated Annealing")
simulated_annealing(problem, T0=0.5, DR=.99999, maxIter=100000)

8-Queens Simulated Annealing
Simulated Annealing intentará resolver el siguiente tablero NQueens: 
  _ _ _ _ _ _ _ _
| * * * * * * Q * |
| Q * * * * * * * |
| * * * * * * Q * |
| * * * * * Q * * |
| * * * * Q * * * |
| * * * * * * Q * |
| * * * * * Q * * |
| Q * * * * * * * |
  _ _ _ _ _ _ _ _

Iteration 0,	Temperaure = 0.5,	Best score = -10,	Current score = -10
Iteration 10000,	Temperaure = 0.45241848280737684,	Best score = 0,	Current score = -2
Iteration 20000,	Temperaure = 0.4093649671714617,	Best score = 0,	Current score = -1
Iteration 30000,	Temperaure = 0.3704085547244124,	Best score = 0,	Current score = 0
Iteration 40000,	Temperaure = 0.33515935269458563,	Best score = 0,	Current score = -1
Iteration 50000,	Temperaure = 0.3032645716895745,	Best score = 0,	Current score = -2
Iteration 60000,	Temperaure = 0.2744049948260552,	Best score = 0,	Current score = -1
Iteration 70000,	Temperaure = 0.24829178286794154,	Best score = 0,	Current score = 0
Iteration 80000,	Temperaure = 0.2246635

# Preguntas:
**1.** Se presenta el tablero: 'queens.txt' de las 8-Queens con función de costo: 
    
        h = número de pares de reinas que se atacan mutuamente    

Además, se implementan los algoritmos Simulating Annealing (SA) y Hill Climbing (HC) con los siguientes parámetros (ellos garantizan una misma cantidad de tableros evaluados como máximo):

        HC: maxIter=1000
        SA: T0=0.5, DR=.99999, maxIter=100000 
        
En el presente laboratorio, se proponen los algoritmos de búsqueda local, los cuales **maximizan** la función. Se solicita modificar el código en ambos algoritmos, con la finalidad de **minimizar** la función de costo (h) **(4 pts)**
                 

**2.** Después de haber completado el código, ¿el algoritmo Hill Climbing presenta soluciones óptimas? ¿Cuáles son las limitaciones que puede presentar este algoritmo de búsqueda local, según los resultados? **(4 pts)**


**3.** En cuanto a las soluciones encontradas por Simulated Annealing, ¿este algoritmo presenta soluciones óptimas? ¿Cómo se pueden interpretar y relacionar los resultados con su teoría y propiedades? Por otro lado, ¿cómo controlamos el grado de exploración de un algoritmo Simulated Annealing? **(4 pts)**


**4.** Usando fundamento teórico, ¿cuáles son las principales ventajas de Simulated Annealing sobre Hill Climbing? Además, relacione su respuesta con los resultados obtenidos en las pruebas. **(4 pts)**



**5.** ¿Qué estrategias de mejora se pueden aplicar en Hill Climbing, en el caso de NQueens? **(2 pts)**




**6.** Justificar, teóricamente, la limitación del método de búsqueda Beam search **(2 pts)** 





Pregunta 2:

El Hill Climbing presenta una solucion optima, la que coincidentemente es la primera. Las limitaciones de este algoritmo es que
es que esta encontrando los maximos locales, es decir analiza los picos de la superficie del espacio de estados donde ningun estado vecino tiene mayor valor. En el caso del caso analizado podemos ver que encuentra una solucion con score 1 y dado que el algoritmo usa un <= esta actualizando este valor para las siguientes iteraciones dado que sus current score son mayores a ese.

Finalmente es importante mencionar que otros de los problemas por los cuales hill-climbing no puede encontrar la solucion optima son las cretas y las mesetas. Las crestas gracias a que existen una secuencia de picos muy dificiles de explorar y las mesetas dado que hill climbing se quedaria perdido ya que son areas planas del espacio de estados.

Pregunta 3:

Dado que el algoritmo Simulated Annealing combina el algoritmo hill-climbing con caminos aleatorios garantiza completitud por lo que podemos asegurar que presentará soluciones optimas. Por otra parte, observando los resultados podemos afirmar lo dicho anteriormente dado que este algoritmo no se basa en los maximos locales sino que nos asegura una completitud en el analisis.
La teoria de este algoritmo nos dice que la temperatura, osea la probabilidad de que estados vecinos con pobre evaluación puedan ser escogidos como sucesores, disminuye con el pasar del tiempo. Esto lo podemos presenciar con la prueba realizada donde observamos como la temperatura va disminuyendo mientras vamos avanzando en las iteraciones analizadas.

El grado de exploracion esta relacionado a la temperatura dado que mientras mayor temperatura tengamos existe una alta probabilidad de que estados vecinos con pobre evaluacion sean escogidos como sucesores. En el algoritmo podemos observar que el parametro DR brindado a la funcion simulated_annealing afecta directamente el grado de exploracion del algoritmo dado que este se esta multiplicando a T(Ti = T0*(DR)^i) lo que aplicaq un decaimiento en la temperatura y por lo tanto se controla el grado de exploracion.

Pregunta 4:

Hill climbing presenta limitaciones que el algoritmo Simulated Annealing logra resolver.
Por ejemplo, Hill Climbing presenta problemas con los maximos locales, es decir en los picos de la superficie del espacio de estados donde ningun estado vecino tiene mayor valor que el estado actual lo que ocasiona que el algoritmo no pueda encontrar una solucion optima. Simulated Annealing resuelve este problema dado que combina el algoritmo Hill Climbing con random walk lo que le garantiza mayor completitud. 
Por otra parte, dado que Simulated Annealing combina esos dos algoritmos le garantiza mayor eficiencia.
Relacionandolo con las pruebas podemos observar que el algoritmo Hill Climbing nos da como respuesta un score de 1 dado que ese es su maximo local y por eso no puede encontrar la solucion optima. Por otro lado, podemos observar que este problema es solucionado por el algoritmo de Simulated Annealing dado que gracias a la combinacion de dos algoritmos mencionados anteriormente nos este dando como resultado una solucion mas optima.

Finalmente es importante mencionar que otros de los problemas por los cuales hill-climbing no puede encontrar la solucion optima son las cretas y las mesetas. Las crestas gracias a que existen una secuencia de picos muy dificiles de explorar y las mesetas dado que hill climbing se quedaria perdido ya que son areas planas del espacio de estados.

Pregunta 5:

Dado que el Hill Climbing presenta el problema de los maximos locales, el cual podemos presenciar en la prueba mostrada dado que se esta atascando en uno ya que nos muestra el mismo resultado podemos realizar posibles mejoras como por ejemplo realizar reinicios aleatorios y de esta forma ejecutar varias busquedas a partir de varios estados iniciales ecogidos aleatoriamente, esta mejora es completa dado que en el peor de los casos terminara generand el estado objetivo como estado inicial pero sin embargo es ineficiente. Otra mejora que podemos realizar es el Stochastic Hill Climbing, el cual escoje aleatoriamente el sucesor con probalilidad proporcional al aumento de la funcion objetivo.

Pregunta 6:

Explicacion previa del funcionamiento del algoritmo:
El metodo Local Beam Search mantiene k estados en lugar de otros para posteriormente emepzar con k estados generados aleatoriamente. En cada interacion generará sucesores y si alguno de estos es objetivo la busqueda termina, de lo contrario elige a los k mejores estaod s de la lista y se continua iterando

Limitacion del metodo Local Beam Search:
Una vez entendido como funciona este algoritmo podemos mencionar su limitacion. El problema con el Local Beam Search es que sufre de falta de diversidad, en otras palabras, la busqueda se concentra rapidamente en una pequeña region del espacio de estados ocasionando que el resultado no sea el optimo en algunas oportunidades.

Como solucionarlo?:
Una alternativa de solucion es el algoritmo Stochastic Beam Search. Este algoritmo escoge k sucesores aleatoriamnete con probabilidad proporcional a sus valores solucionando asi el problema de falta de diversidad mecionado anteriormente.